# Experimentos y análisis


In [1]:
from matplotlib import pyplot as plt
import pandas as pd
import subprocess
import os

CONVEX_HULL_EXEC = "../build/structures"


## Experimento 1
Variar el tamaño del arreglo con `block_size` y `grid_size` fijos

- Párametros:
    - $N$: De $2^5$ a $2^{20}$

In [13]:

rnge = 60000.0
pcnt = 0.0
for i in range(1, 7):
    size = 10**i
    print("Running for size", size)
    # do 10 runs for each size
    for _ in range(10):
        subprocess.run([CONVEX_HULL_EXEC, "1", str(size), str(rnge), str(pcnt)], stdout=subprocess.DEVNULL)
        subprocess.run([CONVEX_HULL_EXEC, "2", str(size), str(rnge), str(pcnt)], stdout=subprocess.DEVNULL)

Running for size 10
Running for size 100
Running for size 1000
Running for size 10000
Running for size 100000
Running for size 1000000


In [2]:
rnge = 60000.0
for i in range(1, 7):
    pcnt = 0.1
    size = 10**i
    print("Running for size", size)
    # do 10 runs for each size
    for _ in range(10):
        for _ in range(10):
            subprocess.run([CONVEX_HULL_EXEC, "3", str(size), str(rnge), str(pcnt)], stdout=subprocess.DEVNULL)
            subprocess.run([CONVEX_HULL_EXEC, "4", str(size), str(rnge), str(pcnt)], stdout=subprocess.DEVNULL)
        pcnt+=0.1

Running for size 10
Running for size 100
Running for size 1000
Running for size 10000


### Análisis

In [ ]:
cpu = pd.read_csv("nbody_naive_cpu.csv", header=None)

naive_cl = pd.read_csv("nbody_naive_cl.csv", header=None)
local_mem_cl = pd.read_csv("nbody_local_mem_cl.csv", header=None)
bidimensional_cl = pd.read_csv("nbody_bidimensional_cl.csv", header=None)

cpu.columns = ["size", "data creation time", "execution time", "total time"]

cl_columns = ["size", "local size", "global size", "local size y", "global size y", "local mem size", "create data time", "copy to device", "execution time", "copy to host"]
naive_cl.columns = cl_columns
local_mem_cl.columns = cl_columns
bidimensional_cl.columns = cl_columns

naive_cl["total time"] = naive_cl["create data time"] + naive_cl["copy to device"] + naive_cl["execution time"] + naive_cl["copy to host"]
local_mem_cl["total time"] = local_mem_cl["create data time"] + local_mem_cl["copy to device"] + local_mem_cl["execution time"] + local_mem_cl["copy to host"]
bidimensional_cl["total time"] = bidimensional_cl["create data time"] + bidimensional_cl["copy to device"] + bidimensional_cl["execution time"] + bidimensional_cl["copy to host"]

# cuda.columns = ["size", "block size", "grid size", "data creation time", "copy to device time", "execution time", "copy to host time", "total time"]

In [ ]:
# plot avg total time for each size
cpu_avg = cpu.groupby("size").mean()
naive_cl_avg = naive_cl.groupby("size").mean()
local_mem_cl_avg = local_mem_cl.groupby("size").mean()
bidimensional_cl_avg = bidimensional_cl.groupby("size").mean()

# cuda_avg = cuda.groupby("size").mean()

fig, ax = plt.subplots()
ax.plot(cpu_avg.index, cpu_avg["total time"], label="CPU")
ax.plot(naive_cl_avg.index, naive_cl_avg["total time"], label="OpenCL-Naive")
ax.plot(local_mem_cl_avg.index, local_mem_cl_avg["total time"], label="OpenCL-Shared Memory")
ax.plot(bidimensional_cl_avg.index, bidimensional_cl_avg["total time"], label="OpenCL-2D")
# ax.plot(cuda_avg.index, cuda_avg["total time"], label="CUDA")
ax.set_xlabel("Size")
ax.set_ylabel("Total time (ms)")
ax.legend()
plt.title("Total time for different sizes")
plt.show()

In [ ]:
# plot avg execution time for each size
fig, ax = plt.subplots()
ax.plot(cpu_avg.index, cpu_avg["execution time"], label="CPU")
ax.plot(naive_cl_avg.index, naive_cl_avg["execution time"], label="OpenCL")
ax.plot(local_mem_cl_avg.index, local_mem_cl_avg["execution time"], label="OpenCL")
ax.plot(bidimensional_cl_avg.index, bidimensional_cl_avg["execution time"], label="OpenCL")
# ax.plot(cuda_avg.index, cuda_avg["execution time"], label="CUDA")
ax.set_xlabel("Size")
ax.set_ylabel("Execution time (ms)")
ax.legend()
plt.title("Execution time for different sizes")
plt.show()

In [ ]:
# print speedup on execution time
speedup_naive_cl = cpu_avg["execution time"] / naive_cl_avg["execution time"]
speedup_shared_cl = cpu_avg["execution time"] / local_mem_cl_avg["execution time"]
speedup_bidimensional_cl = cpu_avg["execution time"] / bidimensional_cl_avg["execution time"]
# speedup_cuda = cpu_avg["execution time"] / cuda_avg["execution time"]

print("Speedup Naive OpenCL")
print(speedup_naive_cl)
print("Speedup Shared Memory OpenCL")
print(speedup_shared_cl)
print("Speedup Bidimensional Mapping OpenCL")
print(speedup_bidimensional_cl)

In [ ]:
cl_avg["data creation time"] /= cl_avg.index
cl_avg["copy to device time"] /= cl_avg.index
cl_avg["execution time"] /= cl_avg.index
cl_avg["copy to host time"] /= cl_avg.index

cuda_avg["data creation time"] /= cuda_avg.index
cuda_avg["copy to device time"] /= cuda_avg.index
cuda_avg["execution time"] /= cuda_avg.index
cuda_avg["copy to host time"] /= cuda_avg.index

# plot a two pie chart of the average time spent on each step for OpenCL and CUDA
fig, ax = plt.subplots(1, 2)
ax[0].pie(cl_avg.iloc[0][3:6], labels=cl_avg.columns[3:6], autopct='%1.1f%%')
ax[0].set_title("OpenCL")
ax[1].pie(cuda_avg.iloc[0][3:6], labels=cuda_avg.columns[3:6], autopct='%1.1f%%')
ax[1].set_title("CUDA")
plt.show()



### Conclusión
- Para arreglos de tamaño menor o igual a $2^{20}$ el rendimiento en CPU y GPU es similar
- Existen un gran speedup en *tiempo de ejecución* no asi en *tiempo total* del programa
- En OpenCL el mayor problema esta en el paso de memoria de CPU a GPU.
- En CUDA también se repite este problema pero esta más equilibrado.